In [1]:
import pandas as pd
import requests
import json
import numpy as np
import time

# Setting  up

In [2]:
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('9wXh5oa4cfW07_eUn5Hu3A', 'm3GnhaEvbM5LGCWX3BMghLCatOLN3g')
# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'babi_research/0.0.1'}
# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'softyarn',
        'password': '1532679!Qa1',
        'User-Agent':'babi_research/0.0.1',
        'refresh_token': '167791397396-1cjKrPIOvr4ct1EFwMm57hY1vi7baw'
        }

In [4]:
# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)
# convert response to JSON and pull access_token value
res.json()

{'access_token': '167791397396-7T0vJkcR0vs85ZAYO-gasU4D2_jmPg',
 'token_type': 'bearer',
 'expires_in': 86400,
 'scope': '*'}

In [5]:
# convert response to JSON and pull access_token value
token = res.json()['access_token']
# add authorization to our headers dictionary
headers['Authorization'] = 'bearer {}'.format(token)
# while the token is valid (~2 hours) we just add headers=headers to our requests. End point: GET /api/v1/me
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers).json()

{'is_employee': False,
 'seen_layout_switch': False,
 'has_visited_new_profile': False,
 'pref_no_profanity': True,
 'has_external_account': False,
 'pref_geopopular': '',
 'seen_redesign_modal': False,
 'pref_show_trending': True,
 'subreddit': {'default_set': False,
  'user_is_contributor': False,
  'banner_img': '',
  'restrict_posting': True,
  'user_is_banned': False,
  'free_form_reports': True,
  'community_icon': None,
  'show_media': True,
  'icon_color': '',
  'user_is_muted': None,
  'display_name': 'u_softyarn',
  'header_img': None,
  'title': '',
  'coins': 0,
  'previous_names': [],
  'over_18': False,
  'icon_size': [256, 256],
  'primary_color': '',
  'icon_img': 'https://styles.redditmedia.com/t5_3ef3ey/styles/profileIcon_snoo86ebddf0-4659-47c8-9ad6-b6ca0588943d-headshot-f.png?width=256&amp;height=256&amp;crop=256:256,smart&amp;s=86680ec95737b7ce10db712e2889672a4de8bbf1',
  'description': '',
  'allowed_media_in_comments': [],
  'submit_link_label': '',
  'header_size

# Miscellanous scraping

In [ ]:
#Get moderators of feminisms
fds_mod= requests.get('https://oauth.reddit.com/r/femaledatingstrategy/about/moderators', headers=headers).json()

In [ ]:
fds_mod_df = pd.DataFrame(fds_mod["data"]["children"])
fds_mod_df

In [ ]:
fds_mod_df.iloc[6,]

# Function

In [6]:
def clean_data(source_dir, comment_bool):
    df = pd.read_csv(source_dir)
    df = df[["id","body"]]
    if comment_bool == True:
        df['id'] ='t1_' + df['id'].astype(str)
    else:
        df['id'] ='t3_' + df['id'].astype(str)
    df["note"] = np.where(df["body"] == "[removed]", "removed", np.where(df["body"] == "[deleted]", "deleted", "text"))
    return df

In [7]:
def get_comments(df) :
    tic = time.time()
    global res_df
    res_df = []
    i = 0
    while i < len(df["id"]):
        j = min(len(df["id"]), i + 40)
        x = ",".join(df["id"][i:j])
        comments = requests.get('https://oauth.reddit.com/api/info',headers=headers,params={'id': x}).json()
        for k in range(j-i):
            comment_df = pd.DataFrame.from_dict(comments.get("data").get("children")[k].get("data"), orient = "index").transpose()
            res_df.append(comment_df)
        i += 40
        if i%300 == 0:
            time.sleep(2)
            #now = time.localtime(time.time())
            print('Time elapsed: ', round(time.time() - tic),' secs, i = ',i)
            #print(time.strftime("Time now: %HH:%M:%S",now),f', i = {i}')
    return pd.concat(res_df)

In [ ]:
def get_submissions(df):
    res_df = pd.DataFrame()
    i = 0
    while i < len(df["id"]):
        x = ",".join(df["id"][i:i+20])
        submissions = requests.get('https://oauth.reddit.com/by_id/names',headers=headers,params={'names': x}).json()
        for k in range(20):
            submission_df = pd.DataFrame.from_dict(comments.get("data").get("children")[k].get("data"), orient = "index").transpose()
            res_df.append(submission_df)
        if i%300 == 0:
            time.sleep(5)
        i += 20  
    return res_df  

# Scraping

# r/femaledatingstrategy

In [ ]:
fds = clean_data("E:/gihub-data/redditbots/fds/fds_comments.csv", comment_bool= True)

In [ ]:
fds_res = get_comments(fds)

In [ ]:
fds_res.to_csv('fds_comment_retrieved.csv')

In [ ]:
requests.get('https://oauth.reddit.com/by_id/names',headers=headers,params={'names': 't3_cipdyg'}).json()

In [ ]:
requests.get('https://oauth.reddit.com/user/')

## r/AskReddit

In [ ]:
askreddit = clean_data("E:/gihub-data/redditbots/control-fds/askreddit.csv", comment_bool= True)

In [ ]:
askreddit_res = get_comments(askreddit)

In [ ]:
askreddit_res.to_csv('E:/gihub-data/redditbots/control-fds/askreddit_comment_retrieved.csv')

# PurplePillDebate

In [ ]:
purplepill = clean_data("E:/gihub-data/redditbots/control-fds/purple.csv", comment_bool= True)

In [ ]:
purple_res = get_comments(purplepill)

In [ ]:
purple_res.to_csv('E:/gihub-data/redditbots/control-fds/purple_comment_retrieved.csv')

# r/AmItheAsshole

In [8]:
AITA = clean_data("E:/gihub-data/redditbots/control-fds/AITA.csv", comment_bool= True)

C:\Users\nguye\AppData\Local\Temp\ipykernel_23996\2879732105.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(source_dir)


In [10]:
AITA1 = AITA.iloc[1:10]

In [11]:
AITA1

,id,body,note
1,t1_f35kgfm,"Mocha’s not that upsetting, though lol",text
2,t1_f35kg4o,So like YTA but I’m not even mad? I’m even ......,text
3,t1_f35kg28,"Pay her back just for that one game, and cut h...",text
4,t1_f35kfy8,"A first impression is hard to break. And yes, ...",text
5,t1_f35kfxm,Joint. Partly to care for the kid considering ...,text
6,t1_f35kfv4,YTA. Hindsight is 20/20. Your parents can only...,text
7,t1_f35kful,I've heard up to 2 years to get a kid back onc...,text
8,t1_f35kfub,Would you know how to Google or call your mom ...,text
9,t1_f35kfu3,NTA why would you even have a drying rack if y...,text


In [ ]:
AITA1 = AITA.iloc[1:10]

In [13]:
AITA_res = get_comments(AITA)

Time elapsed:  8  secs, i =  600
Time elapsed:  16  secs, i =  1200
Time elapsed:  24  secs, i =  1800
Time elapsed:  32  secs, i =  2400
Time elapsed:  39  secs, i =  3000
Time elapsed:  47  secs, i =  3600
Time elapsed:  55  secs, i =  4200
Time elapsed:  63  secs, i =  4800
Time elapsed:  71  secs, i =  5400
Time elapsed:  78  secs, i =  6000
Time elapsed:  86  secs, i =  6600
Time elapsed:  93  secs, i =  7200
Time elapsed:  101  secs, i =  7800
Time elapsed:  109  secs, i =  8400
Time elapsed:  117  secs, i =  9000
Time elapsed:  125  secs, i =  9600
Time elapsed:  133  secs, i =  10200
Time elapsed:  141  secs, i =  10800
Time elapsed:  148  secs, i =  11400
Time elapsed:  156  secs, i =  12000
Time elapsed:  164  secs, i =  12600
Time elapsed:  172  secs, i =  13200
Time elapsed:  180  secs, i =  13800
Time elapsed:  188  secs, i =  14400
Time elapsed:  196  secs, i =  15000
Time elapsed:  203  secs, i =  15600
Time elapsed:  211  secs, i =  16200
Time elapsed:  219  secs, i =  

IndexError: list index out of range

In [15]:
AITA_s = AITA.iloc[49200:49800]